In [1]:
flash2_think_config = {
    "provider": "gemini",
    "model": "gemini-2.0-flash-thinking-exp",
    "limit_requests": 10,
    "limit_tokens": 1_000_000,
    "tools": "code_execution",
    "api_key": "GEMINI_API_KEY"
}

In [2]:
from lionagi import iModel, Branch

flash2_think = iModel(**flash2_think_config)

In [3]:
msgs = [{"role": "user", "content": "Hello, World"}]

In [4]:
a = await flash2_think.invoke(messages=msgs)

03:50:52 - LiteLLM:ERROR: vertex_llm_base.py:279 - Failed to load vertex credentials. Check to see if credentials containing partial/invalid information.
Traceback (most recent call last):
  File "/Users/lion/lionagi/.venv/lib/python3.10/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 275, in _ensure_access_token_async
    self._credentials, cred_project_id = await asyncify(self.load_auth)(
  File "/Users/lion/lionagi/.venv/lib/python3.10/site-packages/litellm/litellm_core_utils/asyncify.py", line 57, in wrapper
    return await anyio.to_thread.run_sync(
  File "/Users/lion/lionagi/.venv/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/Users/lion/lionagi/.venv/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 2461, in run_sync_in_worker_thread
    return await future
  File "/Users/lion/lionagi/.venv/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



In [5]:
a

APICalling(id=f937feca-22b5-4051-bc09-f4237c3506b5, status=failed, duration=5.49561741600337, response=None, error=litellm.APIConnectionError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})
Traceback (most recent call last):
  File "/Users/lion/lionagi/.venv/lib/python3.10/site-packages/litellm/main.py", line 463, in acompletion
    response = await init_response
  File "/Users/lion/lionagi/.venv/lib/python3.10/site-packages/litellm/llms/vertex_ai/gemini/vertex_and_google_ai_studio_gemini.py", line 1030, in async_completion
    _auth_header, vertex_project = await self._ensure_access_token_async(
  File "/Users/lion/lionagi/.venv/lib/python3.10/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 275, in _ensure_access_token_async
    self._credentials, cred_project_id = await asyncify(self.load_auth)(
  File "/Users/lion/lionagi/.venv/lib/python3.10/site-packages/litellm/litellm_core_utils/asyncify.py", line 57, in wrapper
    